In [1]:
import os

if os.getcwd().endswith("notebooks"):
    os.chdir("../")

print("Current working directory: ", os.getcwd())

Current working directory:  /Users/dksifoua/Developer/PycharmProjects/Tokenizers


In [2]:
import re
import tqdm
import itertools
import collections
from typing import List, Tuple

In [3]:
text = """It iterates over the merge_list exactly once, and for each merge, it does one pass of merging (replacing every occurrence of the pair). But note: after merging, the same merge rule might apply again? For example, if we have the merge rule (1,2) -> 3, and the sequence [1,2,1,2], then after one pass we get [3,3]. Then the same merge rule (1,2) doesn't apply anymore. But what if we have a merge rule (3,3) -> 4? Then we would need to apply the next merge rule. However, in the provided code, the merge rule (3,3) would be applied in the next iteration of the for-loop. But note: the merge rule (3,3) is in the list after (1,2). So the provided code does not handle the case where the same merge rule should be applied multiple times? Actually, it doesn't because after the first pass of (1,2) -> 3, the pair (1,2) is gone, so the same merge rule wouldn't be applied again. But what if we have a sequence [1,2,1,2] and we apply the merge rule (1,2) -> 3, we get [3,3]. Then we have a new pair (3,3) that might be in the merge list? But the provided code would then apply the next merge rule for (3,3) if it exists. So the provided code does not require the same merge rule to be applied multiple times because it replaces every occurrence of the pair in one go.
While there are pairs, find the pair that has the highest priority (the one that appears earliest in the merge list) and merge it. Then restart from the beginning of the sequence? Or continue? Actually, they restart from the beginning of the sequence? No, they continue from the next token? Or they restart from the beginning?
While there are pairs that are in the merge list, pick the pair that has the highest priority (the one that appears first in the merge list) and merge it. Then, after merging, it restarts from the beginning of the sequence to check for the next pair? Because after merging, a new token is created and it might form a pair with the token before or after? But the provided code does not restart from the beginning for the same merge rule? Actually, the provided code does not restart at all for the same merge rule. It does one pass for each merge rule.
""".replace(" ", "Ġ").encode(encoding="utf-8")
print(text[:100])

b'It\xc4\xa0iterates\xc4\xa0over\xc4\xa0the\xc4\xa0merge_list\xc4\xa0exactly\xc4\xa0once,\xc4\xa0and\xc4\xa0for\xc4\xa0each\xc4\xa0merge,\xc4\xa0it\xc4\xa0does\xc4\xa0one\xc4\xa0pass\xc4\xa0of'


In [4]:
text_chunks = re.findall(pattern=rb'\xc4\xa0\w+|\xc4\xa0[^\w\s]|\w+|[^\w\s]+|\xc4\xa0', string=text)
print(text_chunks[:100])

[b'It', b'\xc4\xa0iterates', b'\xc4\xa0over', b'\xc4\xa0the', b'\xc4\xa0merge_list', b'\xc4\xa0exactly', b'\xc4\xa0once', b',\xc4\xa0', b'and', b'\xc4\xa0for', b'\xc4\xa0each', b'\xc4\xa0merge', b',\xc4\xa0', b'it', b'\xc4\xa0does', b'\xc4\xa0one', b'\xc4\xa0pass', b'\xc4\xa0of', b'\xc4\xa0merging', b'\xc4\xa0(', b'replacing', b'\xc4\xa0every', b'\xc4\xa0occurrence', b'\xc4\xa0of', b'\xc4\xa0the', b'\xc4\xa0pair', b').\xc4\xa0', b'But', b'\xc4\xa0note', b':\xc4\xa0', b'after', b'\xc4\xa0merging', b',\xc4\xa0', b'the', b'\xc4\xa0same', b'\xc4\xa0merge', b'\xc4\xa0rule', b'\xc4\xa0might', b'\xc4\xa0apply', b'\xc4\xa0again', b'?\xc4\xa0', b'For', b'\xc4\xa0example', b',\xc4\xa0', b'if', b'\xc4\xa0we', b'\xc4\xa0have', b'\xc4\xa0the', b'\xc4\xa0merge', b'\xc4\xa0rule', b'\xc4\xa0(', b'1', b',', b'2', b')\xc4\xa0->\xc4\xa0', b'3', b',\xc4\xa0', b'and', b'\xc4\xa0the', b'\xc4\xa0sequence', b'\xc4\xa0[', b'1', b',', b'2', b',', b'1', b',', b'2', b'],\xc4\xa0', b'then', b'\xc4\xa0after', b'\xc

In [46]:
max_id = 256
id2token = {
    _id: bytes([_id])
    for _id in range(max_id)
}
print(id2token)

{0: b'\x00', 1: b'\x01', 2: b'\x02', 3: b'\x03', 4: b'\x04', 5: b'\x05', 6: b'\x06', 7: b'\x07', 8: b'\x08', 9: b'\t', 10: b'\n', 11: b'\x0b', 12: b'\x0c', 13: b'\r', 14: b'\x0e', 15: b'\x0f', 16: b'\x10', 17: b'\x11', 18: b'\x12', 19: b'\x13', 20: b'\x14', 21: b'\x15', 22: b'\x16', 23: b'\x17', 24: b'\x18', 25: b'\x19', 26: b'\x1a', 27: b'\x1b', 28: b'\x1c', 29: b'\x1d', 30: b'\x1e', 31: b'\x1f', 32: b' ', 33: b'!', 34: b'"', 35: b'#', 36: b'$', 37: b'%', 38: b'&', 39: b"'", 40: b'(', 41: b')', 42: b'*', 43: b'+', 44: b',', 45: b'-', 46: b'.', 47: b'/', 48: b'0', 49: b'1', 50: b'2', 51: b'3', 52: b'4', 53: b'5', 54: b'6', 55: b'7', 56: b'8', 57: b'9', 58: b':', 59: b';', 60: b'<', 61: b'=', 62: b'>', 63: b'?', 64: b'@', 65: b'A', 66: b'B', 67: b'C', 68: b'D', 69: b'E', 70: b'F', 71: b'G', 72: b'H', 73: b'I', 74: b'J', 75: b'K', 76: b'L', 77: b'M', 78: b'N', 79: b'O', 80: b'P', 81: b'Q', 82: b'R', 83: b'S', 84: b'T', 85: b'U', 86: b'V', 87: b'W', 88: b'X', 89: b'Y', 90: b'Z', 91: b'[',

In [47]:
chunks = [*map(lambda text_chunk: tuple(text_chunk), text_chunks)]
print(chunks)

[(73, 116), (196, 160, 105, 116, 101, 114, 97, 116, 101, 115), (196, 160, 111, 118, 101, 114), (196, 160, 116, 104, 101), (196, 160, 109, 101, 114, 103, 101, 95, 108, 105, 115, 116), (196, 160, 101, 120, 97, 99, 116, 108, 121), (196, 160, 111, 110, 99, 101), (44, 196, 160), (97, 110, 100), (196, 160, 102, 111, 114), (196, 160, 101, 97, 99, 104), (196, 160, 109, 101, 114, 103, 101), (44, 196, 160), (105, 116), (196, 160, 100, 111, 101, 115), (196, 160, 111, 110, 101), (196, 160, 112, 97, 115, 115), (196, 160, 111, 102), (196, 160, 109, 101, 114, 103, 105, 110, 103), (196, 160, 40), (114, 101, 112, 108, 97, 99, 105, 110, 103), (196, 160, 101, 118, 101, 114, 121), (196, 160, 111, 99, 99, 117, 114, 114, 101, 110, 99, 101), (196, 160, 111, 102), (196, 160, 116, 104, 101), (196, 160, 112, 97, 105, 114), (41, 46, 196, 160), (66, 117, 116), (196, 160, 110, 111, 116, 101), (58, 196, 160), (97, 102, 116, 101, 114), (196, 160, 109, 101, 114, 103, 105, 110, 103), (44, 196, 160), (116, 104, 101), (

In [48]:
chunk2count = collections.defaultdict(int)
for chunk in chunks:
    chunk2count[chunk] += 1

print(chunk2count)

defaultdict(<class 'int'>, {(73, 116): 2, (196, 160, 105, 116, 101, 114, 97, 116, 101, 115): 1, (196, 160, 111, 118, 101, 114): 1, (196, 160, 116, 104, 101): 43, (196, 160, 109, 101, 114, 103, 101, 95, 108, 105, 115, 116): 1, (196, 160, 101, 120, 97, 99, 116, 108, 121): 1, (196, 160, 111, 110, 99, 101): 1, (44, 196, 160): 19, (97, 110, 100): 5, (196, 160, 102, 111, 114): 7, (196, 160, 101, 97, 99, 104): 2, (196, 160, 109, 101, 114, 103, 101): 22, (105, 116): 3, (196, 160, 100, 111, 101, 115): 6, (196, 160, 111, 110, 101): 6, (196, 160, 112, 97, 115, 115): 4, (196, 160, 111, 102): 8, (196, 160, 109, 101, 114, 103, 105, 110, 103): 4, (196, 160, 40): 14, (114, 101, 112, 108, 97, 99, 105, 110, 103): 1, (196, 160, 101, 118, 101, 114, 121): 2, (196, 160, 111, 99, 99, 117, 114, 114, 101, 110, 99, 101): 2, (196, 160, 112, 97, 105, 114): 8, (41, 46, 196, 160): 2, (66, 117, 116): 6, (196, 160, 110, 111, 116, 101): 2, (58, 196, 160): 2, (97, 102, 116, 101, 114): 2, (116, 104, 101): 7, (196, 160, 

In [49]:
def merge(ids: Tuple[int, ...], pair: Tuple[int, int], new_id: int) -> Tuple[int, ...]:
    if len(ids) < 2:
        return ids

    merged_ids: List[int] = []
    index: int = 0
    while index < len(ids):
        if index < len(ids) - 1 and (ids[index], ids[index + 1]) == pair:
            merged_ids.append(new_id)
            index += 2
            continue

        merged_ids.append(ids[index])
        index += 1

    return tuple(merged_ids)

assert merge(ids=(72, 117, 103, 103, 105, 110), pair=(72, 117), new_id=256) == (256, 103, 103, 105, 110)
assert merge(ids=(72, 117, 103, 103, 105, 110), pair=(117, 103), new_id=256) == (72, 256, 103, 105, 110)
assert merge(ids=(72, 117, 103, 103, 105, 110), pair=(103, 103), new_id=256) == (72, 117, 256, 105, 110)
assert merge(ids=(72, 117, 103, 103, 105, 110), pair=(103, 105), new_id=256) == (72, 117, 103, 256, 110)
assert merge(ids=(72, 117, 103, 103, 105, 110), pair=(105, 110), new_id=256) == (72, 117, 103, 103, 256)
assert merge(ids=(72, 117, 103, 103, 105, 110), pair=(110, 72), new_id=256) == (72, 117, 103, 103, 105, 110)

In [50]:
MAX_VOCAB_SIZE = 500
MIN_TOKEN_FREQUENCY = 2

In [51]:
merge_rules = []
with tqdm.tqdm(total=MAX_VOCAB_SIZE, desc=f"Merging tokens...") as progress_bar:
    while len(id2token) < MAX_VOCAB_SIZE:
        pair2count = collections.defaultdict(int)
        for chunk, count in chunk2count.items():
            for pair in zip(chunk, chunk[1:]):
                pair2count[pair] += count

        top_pair, top_pair_count = max(pair2count.items(), key=lambda item: item[1])
        if top_pair_count < MIN_TOKEN_FREQUENCY:
            break

        new_chunk2count = collections.defaultdict(int)
        for chunk, count in chunk2count.items():
            merged_chunk = merge(ids=chunk, pair=top_pair, new_id=max_id)
            new_chunk2count[merged_chunk] += count

        chunk2count = new_chunk2count
        merge_rules.append((top_pair, max_id))
        id2token[max_id] = id2token[top_pair[0]] + id2token[top_pair[1]]
        max_id += 1

        progress_bar.update()
        progress_bar.set_description(desc=f"Top pair: {top_pair}: {top_pair_count}")

print(merge_rules[:10])

Top pair: (99, 107): 2:  43%|████▎     | 216/500 [00:00<00:00, 2123.31it/s] 

Done.


In [55]:
def encode(ids: Tuple[int, ...], rules: List[Tuple[Tuple[int, int], int]]) -> Tuple[int, ...]:
    for merge_pair, new_id in rules:
        ids = merge(ids=ids, pair=merge_pair, new_id=new_id)

    return ids

In [56]:
encoded_chunk = []
for chunk in chunks:
    encoded_chunk.append(encode(ids=chunk, rules=merge_rules))

print(encoded_chunk[:10])

[(405,), (407, 271), (272, 367), (259,), (266, 95, 326), (384, 347, 116, 368), (272, 304), (270,), (348,), (305,)]


In [64]:
ids = [*itertools.chain(*encoded_chunk)]
print(ids[:10])

[405, 407, 271, 272, 367, 259, 266, 95, 326, 384]


In [66]:
tokens = [*map(id2token.get, ids)]
print(tokens[:10])

[b'It', b'\xc4\xa0iterat', b'es', b'\xc4\xa0o', b'ver', b'\xc4\xa0the', b'\xc4\xa0merge', b'_', b'list', b'\xc4\xa0ex']


In [69]:
print([*map(lambda token: token.decode(encoding="utf-8"), tokens)][:10])

['It', 'Ġiterat', 'es', 'Ġo', 'ver', 'Ġthe', 'Ġmerge', '_', 'list', 'Ġex']
